In [4]:
import numpy as np

# Exercise 3 [Linear Quadratic Regulators]

Here we discuss the problem of computing the optimal control policy for problems of the form
$$ \min_{\mathbf{u}_n} \mathbf{x}_N^T \mathbf{Q}_N \mathbf{x}_N + \sum_{n=0}^{N-1} \mathbf{x}_{n}^T \mathbf{Q}_n \mathbf{x}_n + \mathbf{u}_n^T \mathbf{R}_n \mathbf{u}_n\\
s.t. \ \mathbf{x}_{n+1} = \mathbf{A}_n \mathbf{x}_n + \mathbf{B}_n \mathbf{u}_n + \omega_n$$
where $\mathbf{R}>0$, $\mathbf{Q} \geq 0$ and $\mathbb{E}(\omega_n) = 0$ and $\mathbb{E}(\omega_n^T \omega_n) \leq \infty$ (i.e. the noise has 0 mean and finite variance)

We have seen in the class that the optimal control and optimal value function (and cost-to-go for every stage) could be found by solving the following backward Riccati equations from $N$ to $0$
$$\mathbf{P}_N = \mathbf{Q}_N\\
\mathbf{K}_n = -(\mathbf{R}_n+\mathbf{B}_n^T \mathbf{P}_{n+1} \mathbf{B}_n)^{-1} (\mathbf{B}_n^T \mathbf{P}_{n+1} A_n)\\
\mathbf{P}_n = \mathbf{Q}_n + \mathbf{A}_n^T \mathbf{P}_{n+1} \mathbf{A}_n + \mathbf{A}_n^T \mathbf{P}_{n+1} \mathbf{B}_n \mathbf{K}_n$$

The optimal policy is then $$\mu_n^* = \mathbf{K}_n \mathbf{x}_n$$
and the optimal value function is $$J_0(\mathbf{x}_0) = \mathbf{x}_0^T \mathbf{P}_0 \mathbf{x}_0 + \sum_{n=0}^{N-1} \mathbb{E}(\omega_n^T P_{n+1} \omega_n)$$

## Question 1
Write a function that solves the backward Riccati equations assuming that $A_n$, $B_n$, $Q_n$ and $R_n$ are constant matrices by completing the matrix below

In [5]:
def solve_ricatti_equations(A,B,Q,R,horizon_length):
    """
    This function solves the backward Riccatti equations for regulator problems of the form
    min xQx + sum(xQx + uRu) subject to xn+1 = Axn + Bun
    
    Arguments:
    A, B, Q, R: numpy arrays defining the problem
    horizon_length: length of the horizon
    
    Returns:
    P: list of numpy arrays containing Pn from N to 0
    K: list of numpy arrays containing Kn from N-1 to 0
    """
    P = [1]*horizon_length #will contain the list of Ps from N to 0
    K = [1]*(horizon_length-1) #will contain the list of Ks from N-1 to 0

    N = horizon_length-1
    P[N] = Q
    for n in range(N-1,-1,-1):
        K[n] = -np.linalg.pinv( R+B.T.dot(P[n+1]).dot(B) ).dot(B.T.dot(P[n+1]).dot(A))
        P[n] = Q + A.T.dot(P[n+1]).dot(A) + A.T.dot(P[n+1]).dot(B).dot(K[n])
    
    return P,K

## Question 2

1. Compute the optimal controllers for the four systems shown in Exercise 2 [Controllability] for an horizon length of 100 steps using $Q = I_{3 \times 3}$ and $R = 0.1$.
2. Compute the behavior of each system for initial conditions $x_0 = [1,1,1]$ when using the optimal controller and compare the system behavior when no controller is used ($u_n = 0$). 
3. For which system did the controller lead to a stable system?

In [6]:
length = 100;
Q = np.mat([ [1,0,0],[0,1,0],[0,0,1] ])
R = 0.1
x0 = np.mat([1,1,1]).T

A1 = np.mat([ [1,0,1],[0,1.5,0],[1,0,0] ])
B1 = np.mat([ 0,0,1 ]).T
[ P,K ] = solve_ricatti_equations(A1,B1,Q,R,length)
J0_x0 = x0.T.dot(P[0]).dot(x0)
[ P,K ] = solve_ricatti_equations(A1,np.mat([ 0,0,0 ]).T,Q,R,length)
J0_x0_uis0 = x0.T.dot(P[0]).dot(x0)
print(J0_x0,J0_x0_uis0)

A2 = np.mat([ [1,0,1],[0,1.5,0],[1,0,0] ])
B2 = np.mat([ 0,1,1 ]).T
[ P,K ] = solve_ricatti_equations(A2,B2,Q,R,length)
J0_x0 = x0.T.dot(P[0]).dot(x0)
# print K
[ P,K ] = solve_ricatti_equations(A2,np.mat([ 0,0,0 ]).T,Q,R,length)
J0_x0_uis0 = x0.T.dot(P[0]).dot(x0)
print(J0_x0,J0_x0_uis0)


A3 = np.mat([ [0.5,0,0.5],[0,-0.5,-1],[0,0,0.5] ])
B3 = np.mat([ 1,0,1 ]).T
[ P,K ] = solve_ricatti_equations(A3,B3,Q,R,length)
J0_x0 = x0.T.dot(P[0]).dot(x0)
[ P,K ] = solve_ricatti_equations(A3,np.mat([ 0,0,0 ]).T,Q,R,length)
J0_x0_uis0 = x0.T.dot(P[0]).dot(x0)
print(J0_x0,J0_x0_uis0)

A4 = np.mat([ [0.5,0.5,0],[0,-0.5,-1],[-0.1,0,0.5] ])
B4 = np.mat([ 0,1,0 ]).T
[ P,K ] = solve_ricatti_equations(A4,B4,Q,R,length)
J0_x0 = x0.T.dot(P[0]).dot(x0)
[ P,K ] = solve_ricatti_equations(A4,np.mat([ 0,0,0 ]).T,Q,R,length)
J0_x0_uis0 = x0.T.dot(P[0]).dot(x0)
print(J0_x0,J0_x0_uis0)




(matrix([[1.32233593e+35]]), matrix([[7.34544999e+41]]))
(matrix([[1774.14276921]]), matrix([[7.34544999e+41]]))
(matrix([[6.1375247]]), matrix([[7.76296296]]))
(matrix([[4.5572392]]), matrix([[6.71582696]]))
